In [2]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split

In [3]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x1 = df.iloc[:, 0].values
        self.x2 = df.iloc[:, 1].values
        self.x3 = df.iloc[:, 2].values
        self.y = df.iloc[:, 3].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x1[index], self.x2[index], self.x3[index]])
        y = torch.FloatTensor([int(self.y[index])])
        return x, y
    
    def __len__(self):
        return self.length

In [4]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(3, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layer(x)
        return x

In [5]:
dataset = CustomDataset('./datasets/binary.csv')
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size], torch.manual_seed(4))
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)

lr = 0.0001

optimizer = optim.SGD(model.parameters(), lr=lr)

In [8]:
epochs = 10000

for epoch in range(epochs):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f'Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}')

Epoch : 1000, Model : [Parameter containing:
tensor([[0.0034, 0.0005, 0.0041]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.0346], device='cuda:0', requires_grad=True)], Cost : 0.666
Epoch : 2000, Model : [Parameter containing:
tensor([[0.0045, 0.0017, 0.0051]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.1609], device='cuda:0', requires_grad=True)], Cost : 0.652
Epoch : 3000, Model : [Parameter containing:
tensor([[0.0052, 0.0017, 0.0053]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.2843], device='cuda:0', requires_grad=True)], Cost : 0.636
Epoch : 4000, Model : [Parameter containing:
tensor([[0.0059, 0.0036, 0.0064]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.4046], device='cuda:0', requires_grad=True)], Cost : 0.627
Epoch : 5000, Model : [Parameter containing:
tensor([[0.0065, 0.0034, 0.0068]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.5220], de

In [9]:
with torch.no_grad():
    model.eval()
    for x, y in validation_dataloader:
        x = x.to(device)
        y = y.to(device)

        outputs = model(x)

        print(outputs)
        print(outputs >= torch.FloatTensor([0.5]).to(device))
        print('--------------------')

tensor([[0.5212],
        [0.6681],
        [0.3523],
        [0.5635]], device='cuda:0')
tensor([[ True],
        [ True],
        [False],
        [ True]], device='cuda:0')
--------------------
tensor([[0.5152],
        [0.5849],
        [0.5186],
        [0.6749]], device='cuda:0')
tensor([[True],
        [True],
        [True],
        [True]], device='cuda:0')
--------------------
tensor([[0.6119],
        [0.5776],
        [0.4184],
        [0.6082]], device='cuda:0')
tensor([[ True],
        [ True],
        [False],
        [ True]], device='cuda:0')
--------------------
tensor([[0.3567],
        [0.5791],
        [0.6089],
        [0.6064]], device='cuda:0')
tensor([[False],
        [ True],
        [ True],
        [ True]], device='cuda:0')
--------------------
tensor([[0.5743],
        [0.6031],
        [0.3831],
        [0.6581]], device='cuda:0')
tensor([[ True],
        [ True],
        [False],
        [ True]], device='cuda:0')
--------------------
tensor([[0.5724],
 